# How Many Games Does It Take?

Investigating the effect that the number of games per iteration has on population fitness.

## Hypothesis

We know that increasing the number of games each player must play will give each player a chance to compete against a wider subset of strategies.

So let us hypothesize that increasing the number of games will give us more consistent fitness results.

## What we will be testing

Game: We will run our tests using Competitive Graph Coloring and our 14-node graph from a previous Notebook.

Parameters:
* Our fitness threshold will start at 0.5 and will not increase for the duration of our tests
* The population size will remain at 100.
* The number of iterations will remain at 10.
* Mutation rate will remain at 0.025 for all tests.
* Our control group will require at minimum 10 games to be played.
* Our bounds will be fixed at 3. We know this graph has a known game chromatic color of 4, and that it is *possible* to win with 3 colors.

So, the *only* parameter that will be tested is the minimum number of games played.

## Setup

We need to import the necessary files and create a graph and ruleset.

In [1]:
import sys
sys.path.append("../../simple_games")

from classes.genetic_algorithm import GeneticAlgorithm

# Import the rulesets and strategy
from graph_coloring.classes.gc_ruleset import GCRuleset
from graph_coloring.classes.gc_random_init_strategy import GCRandomInitStrategy

In [2]:
# Create a graph to play on
initial_state = [
    {"color": 0, "adj": [5]},
    {"color": 0, "adj": [6]},
    {"color": 0, "adj": [7]},
    {"color": 0, "adj": [8]},
    {"color": 0, "adj": [5]},
    {"color": 0, "adj": [0,4,6,10]},
    {"color": 0, "adj": [1,5,7,11]},
    {"color": 0, "adj": [2,6,8,12]},
    {"color": 0, "adj": [3,7,9,13]},
    {"color": 0, "adj": [8]},
    {"color": 0, "adj": [5]},
    {"color": 0, "adj": [6]},
    {"color": 0, "adj": [7]},
    {"color": 0, "adj": [8]},
]

In [3]:
# Ruleset of the game being played on
ruleset = GCRuleset("Graph Coloring Ruleset", initial_state, bounds = 3)

Let's also define some functions to allow us run a test and see information about the player populations.

In [4]:
def print_pop(pop, fitness = 0.5):
    """
    Displays individuals in the population whose fitness is at or above the provided threshold.
    
    Args:
        pop : Population to display
        fitness (Optional) : Minimum fitness threshold
    """
    for p in pop:
        if p.fitness() >= fitness:
            print(p)

            
            
def percent_good_players(pop, fitness = 0.5):
    """
    Prints the percentage of a population that is above a specified fitness.
    
    Args:
        pop : Population to analyze
        fitness (Optional) : Minimum fitness threshold
    """
    good_pop = [p for p in pop if p.fitness() >= fitness]
    percent = len(good_pop) / len(pop)
    print("{:.2f}% above fitness {}".format(percent * 100, fitness))
    
    
    
def fitness_thresholds(p1_pop, p2_pop):
    """
    Displays the fitness thresholds for each population.
    
    Args:
        p1_pop : Player 1 population
        p2_pop : Player 2 population
    """
    print("Player 1 Fitness Thresholds:")
    for i in range(0, 11, 1):
        percent_good_players(p1_pop, fitness=i/10)

    print("\nPlayer 2 Fitness Thresholds:")
    for i in range(0, 11, 1):
        percent_good_players(p2_pop, fitness=i/10)

        
        
def run_test(num_games):
    """
    Runs a series of genetic algorithms, averaging the results.
    
    Args:
        num_games : Minimum number of games each player must play per iteration.
    """
    total_p1_pop = []
    total_p2_pop = []
    
    # Run 10 tests
    for i in range(10):
        # Create a new Evolution instance with the example strategy
        test_group = GeneticAlgorithm(
            ruleset,
            # Random-on-initialization strategy for generating populations of random players
            random_on_init_strat = GCRandomInitStrategy,
            # Data to be used by the above strategy
            strat_data = {"vertices": range(len(ruleset.initial_state)), "colors": range(1, ruleset.bounds + 1)},
            # Size of the populations
            pop_size = 100,
            # Number of generations to iterate through
            iterations = 10,
            # Minimum number of games each player must play during a generation
            num_games = num_games,
            # Starting fitness threshold
            fitness = 0.5,
            # Maximum fitness threshold; be careful of setting this too close to 1.0
            max_fitness = 0.5,
            # How much the fitness threshold should increment after each iteration
            fitness_increment = 0.0,
            # Chance of a mutation to occur during player reproduction
            mutation_rate = 0.025
        )
    
        # Run the algorithm
        p1_pop, p2_pop = test_group.evolve()
        
        # Append to populations
        total_p1_pop.extend(p1_pop)
        total_p2_pop.extend(p2_pop)
    
    fitness_thresholds(total_p1_pop, total_p2_pop)

## Control Group

Our control group will require each player to play at least 10 games per iteration. As stated earlier, fitness, population size, and number of iterations will remain fixed.

In [5]:
run_test(num_games=10)

Player 1 Fitness Thresholds:
100.00% above fitness 0.0
95.70% above fitness 0.1
95.70% above fitness 0.2
95.70% above fitness 0.3
95.60% above fitness 0.4
95.10% above fitness 0.5
86.60% above fitness 0.6
76.00% above fitness 0.7
61.90% above fitness 0.8
48.40% above fitness 0.9
29.80% above fitness 1.0

Player 2 Fitness Thresholds:
100.00% above fitness 0.0
7.20% above fitness 0.1
7.20% above fitness 0.2
7.20% above fitness 0.3
7.20% above fitness 0.4
7.20% above fitness 0.5
1.60% above fitness 0.6
0.20% above fitness 0.7
0.10% above fitness 0.8
0.00% above fitness 0.9
0.00% above fitness 1.0


The Player 1 population is doing very well. The Player 2 population is not doing nearly as well.
* Majority of P1 (61.9%) was at or above 0.8, 29.8% were perfect players.
* Only 0.1% of P2 was at or above 0.0, 7.2% were below 0.5.

## Test 1: Decreasing Number of Games

Here we will examine the effects of decreasing the number of games. Let's do a simple test of halving them to 5.

In [6]:
run_test(num_games=5)

Player 1 Fitness Thresholds:
100.00% above fitness 0.0
94.90% above fitness 0.1
94.90% above fitness 0.2
94.90% above fitness 0.3
94.90% above fitness 0.4
93.80% above fitness 0.5
82.10% above fitness 0.6
68.70% above fitness 0.7
55.30% above fitness 0.8
40.90% above fitness 0.9
28.40% above fitness 1.0

Player 2 Fitness Thresholds:
100.00% above fitness 0.0
9.40% above fitness 0.1
9.40% above fitness 0.2
9.40% above fitness 0.3
9.40% above fitness 0.4
9.40% above fitness 0.5
1.70% above fitness 0.6
0.70% above fitness 0.7
0.10% above fitness 0.8
0.00% above fitness 0.9
0.00% above fitness 1.0


There seems to be a slight decline in the performance of the Player 1 population and a slight improvement for Player 2.
* Majority of P1 (55.3%) were at or above 0.8. 28.4% were perfect players.
* 9.4% of P2 was above 0.5, but only %0.1 was above 0.8.

## Test 2: Increasing Number of Games

Here we will examine the effects of increasing the number of games. Let's do a simple test of doubling them to 20.

In [7]:
run_test(num_games=20)

Player 1 Fitness Thresholds:
100.00% above fitness 0.0
93.50% above fitness 0.1
93.50% above fitness 0.2
93.50% above fitness 0.3
93.30% above fitness 0.4
91.30% above fitness 0.5
81.10% above fitness 0.6
65.50% above fitness 0.7
49.10% above fitness 0.8
35.90% above fitness 0.9
19.20% above fitness 1.0

Player 2 Fitness Thresholds:
100.00% above fitness 0.0
6.90% above fitness 0.1
6.90% above fitness 0.2
6.90% above fitness 0.3
6.90% above fitness 0.4
6.90% above fitness 0.5
0.40% above fitness 0.6
0.00% above fitness 0.7
0.00% above fitness 0.8
0.00% above fitness 0.9
0.00% above fitness 1.0


We see declining performance in the Player 1 population across the board, and likewise for Player 2.
* 19.2% of P1 were perfect players, with nearly the majority (49.1%) being above 0.8.
* 6.9% of P2 were above 0.5, but only 0.4% were above 0.6.

## Test 4: Increasing Number of Games Again

Here we will examine the effects of increasing the number of games. Let's do a more significant 50 games.

In [8]:
run_test(num_games=50)

Player 1 Fitness Thresholds:
100.00% above fitness 0.0
96.10% above fitness 0.1
96.10% above fitness 0.2
96.10% above fitness 0.3
96.10% above fitness 0.4
94.30% above fitness 0.5
82.80% above fitness 0.6
68.00% above fitness 0.7
51.60% above fitness 0.8
31.10% above fitness 0.9
10.50% above fitness 1.0

Player 2 Fitness Thresholds:
100.00% above fitness 0.0
17.90% above fitness 0.1
17.90% above fitness 0.2
17.90% above fitness 0.3
17.90% above fitness 0.4
17.90% above fitness 0.5
0.00% above fitness 0.6
0.00% above fitness 0.7
0.00% above fitness 0.8
0.00% above fitness 0.9
0.00% above fitness 1.0


Both populations had a higher number of lower-fitness players and a lower number of higher-fitness players.
* Majority of P1 (51.6%) were at or above 0.8. 10.5% were perfect players.
* 17.9% of P2 were at or above 0.5. None were higher.

## Test 4: Increasing Number of Games Incredibly

Now let's see what an order of magnitude increase yields. We will test with 100 games.

In [9]:
run_test(num_games=100)

Player 1 Fitness Thresholds:
100.00% above fitness 0.0
96.50% above fitness 0.1
96.50% above fitness 0.2
96.50% above fitness 0.3
96.50% above fitness 0.4
94.40% above fitness 0.5
87.20% above fitness 0.6
72.40% above fitness 0.7
51.20% above fitness 0.8
31.60% above fitness 0.9
9.30% above fitness 1.0

Player 2 Fitness Thresholds:
100.00% above fitness 0.0
10.40% above fitness 0.1
10.40% above fitness 0.2
10.40% above fitness 0.3
10.40% above fitness 0.4
10.40% above fitness 0.5
0.00% above fitness 0.6
0.00% above fitness 0.7
0.00% above fitness 0.8
0.00% above fitness 0.9
0.00% above fitness 1.0


Identical format as last results- both populations had a higher number of worse players and a lower number of better players.
* Majority of P1 (51.2%) were at 0.8. 9.3% were perfect players.
* 10.4% of P2 were at 0.5. None were higher.

## Conclusion

Decreasing the number of games from 10 to 5 is not a good idea. Players need to be abe 